In [1]:
!pip install langchain
!pip install chromadb

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from config import openaiapikey as key

In [3]:
import os
os.environ["OPENAI_API_KEY"] = key

In [4]:
template = """ You are going to be my assistant.
Please try to give me the most beneficial answers to my
question with reasoning for why they are correct.

 Question: {input} Answer: """
prompt = PromptTemplate(template=template, input_variables=["input"])

In [5]:
chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0.8))

In [6]:
embeddings = OpenAIEmbeddings()

In [7]:
file = open("./sample_blog.txt", "r")
text = file.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)


In [8]:
# pass the text and embeddings to ChromaDB
db = Chroma.from_texts(texts, embeddings,metadatas=[{"source": str(i)} for i in range(len(texts))])

Using embedded DuckDB without persistence: data will be transient


In [9]:
my_chain = load_qa_with_sources_chain(OpenAI(temperature=0.8), chain_type="refine", verbose=True)
query = "All places in Kochi"
documents = db.similarity_search(query)
que = "Make an itinerary with all the places mentioned along with their descriptions in one or two sentences."
result = my_chain({"input_documents": documents, "question": que})
result



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Context information is below. 
---------------------
Content: By the end of the day, enjoy watching or participate in kolam making at the Brahmin Hindu Agraharam. Kolam is a colorful South Indian drawing made using rice flour, rock powder and color powder often drawn on the floor of Hindu homes by female members. Ride back to the ship with memories and beautiful photographs of this rich day. Book now to savor the rich culture of Kochi.

The tourist places in Kochi are absolutely captivating and are sure to leave you with a bag full of beautiful memories and aspirations. For those who wish to explore the real treasures of Kerala, a visit to Kochi tourist places is a must.
Source: 11
---------------------
Given the context information and not prior knowledge, answer the question: Make an itinerary with all the places mentioned along with their descriptions in one or two sentences.



{'input_documents': [Document(page_content='By the end of the day, enjoy watching or participate in kolam making at the Brahmin Hindu Agraharam. Kolam is a colorful South Indian drawing made using rice flour, rock powder and color powder often drawn on the floor of Hindu homes by female members. Ride back to the ship with memories and beautiful photographs of this rich day. Book now to savor the rich culture of Kochi.\n\nThe tourist places in Kochi are absolutely captivating and are sure to leave you with a bag full of beautiful memories and aspirations. For those who wish to explore the real treasures of Kerala, a visit to Kochi tourist places is a must.', metadata={'source': '11'}),
  Document(page_content='Bolghatty Island is a short boat ride away from mainland Kochi and has Bolghatty Palace which is one of the oldest existing Dutch palaces outside Holland. Bolghatty Palace is now converted into a hotel and there is even a small golf course, a swimming pool, 9-hole golf course, ayu

In [10]:
print(result['output_text'])



Morning: Visit the Brahmin Hindu Agraharam to witness the traditional Kolam making ritual. Afternoon: Explore Kochi's captivating and scenic tourist spots, including Bolghatty Island, with its Bolghatty Palace, golf course, swimming pool, 9-hole golf course, ayurvedic center, and daily Kathakali performances. Stop by Fort Kochi, home to the famous Chinese fishing nets, St Francis Church, and Vasco House. Then, head to Mattancherry to visit its historic palace and synagogue. Continue on to the nearby town of Malayattoor to explore the Malayatoor Church on top of the 609 m high Malayatoor Hill and Pallipuram Fort, the oldest surviving fort in India built by European settlers. Lastly, stop by Cherai and Vypeen beach to enjoy the view. End the day with some shopping at the Lulu Shopping Mall in Edappally with 300+ food outlets, stores, restaurants, and international brands. Evening: Ride back to the ship with photos and memories of the day. 

Sources:
1. https://www.thehindu.com/society/